In [1]:
#include <iostream>
#include <thread>
#include<functional> 

In [2]:
namespace n1 {
    void f(int i, std::string const& s)
    {
        // just for delay
        long sum = 0;
        for (long i = 0; i < 9000000; ++i) {
            sum += s.length();
        }

        std::cout << i << ", " << s << ", " << sum << "\n";
    }
    std::thread t(f, 3, "hello");
}

3, hello, 45000000


In [3]:
{
    using namespace n1;
    int some_param = 10;

    char buffer[64]; // 1
    sprintf(buffer, "%i", some_param);

    std::thread t(f, 3, buffer); // 2
    t.detach();
    buffer[0] = 'A';
    // 运行到这里，函数返回，buffer立刻被回收，而thread可能没有结束，所以buffer里面可能是任何值
}

3, A0, 18000000


In [4]:
{
    using namespace n1;
    int some_param = 10;

    char buffer[64]; // 1
    sprintf(buffer, "%i", some_param);

    // 解决方法，使用std::string，避免悬垂指针
    std::thread t2(f, 3, std::string(buffer));
    t2.detach();
    buffer[0] = 'B';
}

3, 10, 18000000


还可能遇到相反的情况：期望传递一个引用，但整个对象被复制了。当线程更新一个引用传递的数据结构时，这种情况就可能发生，比如：

In [5]:
namespace n2 {
    using widget_id = int;
    struct widget_data {
        int data{};
    };

    void update_data_for_widget(widget_id w, widget_data& data) // 1
    {
        data.data++;
        std::cout << "update_data_for_widget(): data = " << data.data << "\n";
    }
    void display_status()
    {
        std::cout << "display_status() \n";
    }
    void process_widget_data(widget_data& data)
    {
        std::cout << "process_widget_data() \n";
    }
}

In [6]:
/*
{
    using namespace n2;

    widget_id w = 1;
    widget_data data;
    std::thread t(update_data_for_widget, w, data); // 2
    display_status();
    t.join();
    process_widget_data(data); // 3
}
*/

虽然update_data_for_widget①的第二个参数期待传入一个引用，但是	std::thread	的构造函 数②并不知晓；构造函数无视函数期待的参数类型，并盲目的拷贝已提供的变量。当线程调用 update_data_for_widget函数时，传递给函数的参数是data变量内部拷贝的引用，而非数据本 身的引用。因此，当线程结束时，内部拷贝数据将会在数据更新阶段被销毁，且 process_widget_data将会接收到没有修改的data变量③。**可以使用	std::ref	将参数转换成引 用的形式，从而可将线程的调用改为以下形式**：

In [7]:
{
    using namespace n2;

    widget_id w = 1;
    widget_data data;
    std::thread t(update_data_for_widget, w, std::ref(data)); // 2
    display_status();
    t.join();
    process_widget_data(data); // 3
}

display_status() 
update_data_for_widget(): data = 1
process_widget_data() 


如果你熟悉	std::bind	，就应该不会对以上述传参的形式感到奇怪，因为	std::thread	构造函 数和	std::bind	的操作都在标准库中定义好了，可以传递一个成员函数指针作为线程函数， 并提供一个合适的对象指针作为第一个参数：

In [8]:
{
    class X
    {
    public:
        void do_lengthy_work()
        {
            std::cout << "X::do_lengthy_work() \n";
        }
    };

    X my_x;
    std::thread t(&X::do_lengthy_work, &my_x); // 1
    t.join();
}

X::do_lengthy_work() 
